In [2]:
import requests 
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go

In [23]:
polling_average = requests.get('https://projects.fivethirtyeight.com/polls/president-general/2024/national/polling-average.json').json()
df_polling_average = pd.DataFrame(polling_average)


polls = requests.get('https://projects.fivethirtyeight.com/polls/president-general/2024/national/polls.json').json()
df_polls = pd.DataFrame(polls)

In [21]:
df_polls.columns

Index(['id', 'subgroup', 'sampleSize', 'population', 'url', 'created_at',
       'startDate', 'endDate', 'pollster', 'pollsterRatingLink',
       'partisan_pollster', 'answers', 'type', 'seat_name', 'tracking',
       'headToHead', 'sponsors', 'internal', 'inAvg', 'cycle', 'partisan',
       'state', 'path', 'hasAvg'],
      dtype='object')

In [21]:
import requests 
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go

polling_average = requests.get('https://projects.fivethirtyeight.com/polls/president-general/2024/national/polling-average.json').json()
df_polling_average = pd.DataFrame(polling_average)

fig = px.line(df_polling_average, x ='date', y='pct_estimate', color='candidate', error_x='hi', title='EUA 2024 - Presidencial Polls' )
fig.update_layout(hovermode="x")
fig.show()

In [29]:
import requests 
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go

# polling_average = requests.get('https://projects.fivethirtyeight.com/polls/president-general/2024/national/polling-average.json').json()
# df_polling_average = pd.DataFrame(polling_average)

df_polling_average =pd.read_json('https://projects.fivethirtyeight.com/polls/president-general/2024/national/polling-average.json')
df = df_polling_average[df_polling_average["candidate"] != "Kennedy" ] .copy()

# Preparar os dados
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values(by='date')

# Figura
fig = go.Figure()

# Dicionário de cores
colors = {
    'Trump': 'blue',
    'Harris': 'red',
}

# Plotar para cada candidato
for candidate_name, candidate_data in df.groupby('candidate'):
    candidate_data = candidate_data.sort_values(by='date')
    
    x = candidate_data['date']
    y = candidate_data['pct_estimate']
    y_upper = candidate_data['hi']
    y_lower = candidate_data['lo']
    
    # Linha principal
    fig.add_trace(go.Scatter(
        x=x,
        y=y,
        mode='lines',
        name=f'{candidate_name}',
        line=dict(color=colors.get(candidate_name, 'black'))
    ))
    
    # Preenchimento entre os limites
    fig.add_trace(go.Scatter(
        x=pd.concat([x, x[::-1]]),
        y=pd.concat([y_upper, y_lower[::-1]]),
        fill='toself',
        fillcolor=colors.get(candidate_name, 'black'),
        opacity=0.1,
        line=dict(color='rgba(255,255,255,0)'),
        hoverinfo="skip",
        showlegend=False
    ))

# Layout
fig.update_layout(
    title='Estimativas com Limites Inferior e Superior por Candidato',
    xaxis_title='Data',
    yaxis_title='Estimativa (%)',
    template='plotly_white',
    hovermode='x',
)

# Exibir
fig.show()


In [30]:
# import requests 
# import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

# polling_average = requests.get('https://projects.fivethirtyeight.com/polls/president-general/2024/national/polling-average.json').json()
# df_polling_average = pd.DataFrame(polling_average)

df_polling_average =pd.read_json('https://projects.fivethirtyeight.com/polls/president-general/2024/national/polling-average.json')
df = df_polling_average[df_polling_average["candidate"] != "Kennedy" ] .copy()

# Pivotar o DataFrame
df_pivot = df.pivot(index='date', columns='candidate', values='pct_estimate').reset_index()
df_pivot['Difference'] = df_pivot['Harris'] - df_pivot['Trump']

# Pivotar 'hi' e 'lo' e mesclar
hi_pivot = df.pivot(index='date', columns='candidate', values='hi').reset_index()
lo_pivot = df.pivot(index='date', columns='candidate', values='lo').reset_index()
df_pivot = df_pivot.merge(hi_pivot, on='date', suffixes=('', '_hi'))
df_pivot = df_pivot.merge(lo_pivot, on='date', suffixes=('', '_lo'))

# Criar o gráfico
fig = go.Figure()

# Harris
fig.add_trace(go.Scatter(
    x=df_pivot['date'],
    y=df_pivot['Harris'],
    mode='lines',
    name='Harris',
    line=dict(color='red'),
    hovertemplate=(
        '<b>Harris</b><br>' +
        'Data: %{x|%d-%m-%Y}<br>' +
        'Estimativa: %{y:.2f}%<br>' +
        'Diferença (Harris - Trump): %{customdata:.2f}%<extra></extra>'
    ),
    customdata=df_pivot['Difference']
))

fig.add_trace(go.Scatter(
    x=pd.concat([df_pivot['date'], df_pivot['date'][::-1]]),
    y=pd.concat([df_pivot['Harris_hi'], df_pivot['Harris_lo'][::-1]]),
    fill='toself',
    fillcolor='rgba(255, 0, 0, 0.1)',
    line=dict(color='rgba(255,255,255,0)'),
    hoverinfo='skip',
    showlegend=False
))

# Trump
fig.add_trace(go.Scatter(
    x=df_pivot['date'],
    y=df_pivot['Trump'],
    mode='lines',
    name='Trump',
    line=dict(color='blue'),
    hovertemplate=(
        '<b>Trump</b><br>' +
        'Data: %{x|%d-%m-%Y}<br>' +
        'Estimativa: %{y:.2f}%<br>' +
        'Diferença (Harris - Trump): %{customdata:.2f}%<extra></extra>'
    ),
    customdata=df_pivot['Difference']
))

fig.add_trace(go.Scatter(
    x=pd.concat([df_pivot['date'], df_pivot['date'][::-1]]),
    y=pd.concat([df_pivot['Trump_hi'], df_pivot['Trump_lo'][::-1]]),
    fill='toself',
    fillcolor='rgba(0, 0, 255, 0.1)',
    line=dict(color='rgba(255,255,255,0)'),
    hoverinfo='skip',
    showlegend=False
))

# Layout
fig.update_layout(
    title='Estimativas com Diferença entre Harris e Trump',
    xaxis_title='Data',
    yaxis_title='Estimativa (%)',
    template='plotly_white',
    hovermode='x',
)

# Exibir
fig.show()


In [28]:
df_pivot

candidate,date,Harris,Trump,Difference,Harris_hi,Trump_hi,Harris_lo,Trump_lo
0,2024-07-24,44.87105,44.03770,0.83335,46.147110,45.450462,43.569825,42.671428
1,2024-07-25,44.63285,44.24270,0.39015,45.854837,45.415640,43.497355,43.049063
2,2024-07-26,44.15055,43.92505,0.22550,45.262408,44.979538,42.965415,42.848920
3,2024-07-27,44.21675,43.91775,0.29900,45.435408,45.043703,43.049853,42.709105
4,2024-07-28,44.29690,44.07805,0.21885,45.448500,45.234825,43.147647,42.926135
5,2024-07-29,44.40710,43.97495,0.43215,45.560203,45.139722,43.182365,42.873560
6,2024-07-30,44.54535,43.63010,0.91525,45.723430,44.656750,43.464315,42.612733
7,2024-07-31,44.71760,43.50510,1.21250,45.950938,44.631628,43.687092,42.412872
8,2024-08-01,44.86960,43.65805,1.21155,46.040363,44.751015,43.757100,42.656912
9,2024-08-02,45.03665,43.55770,1.47895,46.172208,44.637662,43.912202,42.478670
